In [1]:
import prettytensor as pt
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from datetime import timedelta
import time
import math

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


# DATA IMPORT

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST/',one_hot=True)

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


In [3]:
data.test.cls = np.argmax(data.test.labels ,axis =1)

# ----------------------------MODEL----------------------------

## MODEL INPUTS

Images.

In [4]:
x = tf.placeholder(tf.float32,shape = [None,784],name='x')
x_image = tf.reshape(x,[-1,28,28,1])
#                   [num_images , img_width , img_height , channels]
x_pretty = pt.wrap(x_image)

Labels

In [5]:
y_true = tf.placeholder(tf.float32 , shape = [None,10],name='y')
y_true_cls = tf.argmax(y_true,dimension=1)

## MODEL

In [6]:
with pt.defaults_scope(activation_fn=tf.nn.relu):
    y_pred ,loss = x_pretty.\
        conv2d(kernel=5,depth=16,name='layer_conv1').\
        max_pool(kernel=2,stride=2).\
        conv2d(kernel=5,depth=36,name='layer_conv2').\
        max_pool(kernel=2,stride=2).\
        flatten().\
        fully_connected(size=128,name='layer_fc1').\
        softmax_classifier(num_classes=10,labels=y_true)  
        
y_pred_cls = tf.argmax(y_pred,dimension=1)

In [7]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss)
correct_prediction = tf.equal(y_pred_cls,y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

### MODEL FUNCTIONS

In [8]:
train_batch_size = 128
total_iterations = 0
def optimize(num_iterations):
    global total_iterations
    start_time = time.time()
    for i in range(total_iterations,total_iterations+num_iterations) :
        x_batch,y_true_batch = data.train.next_batch(train_batch_size)
        feed_dict_train = {x:x_batch,y_true:y_true_batch}
        session.run(optimizer,feed_dict=feed_dict_train)
        
        if i%100 == 0:
            acc = session.run(accuracy,feed_dict=feed_dict_train)
            print 'Accuracy @iter {0:>6} , Training Acc {1}'.format(i+1,acc)
            
    total_iterations += num_iterations
    end_time = time.time()
    print 'Total Time Elapsed :',round(end_time-start_time,3)

### OTHER FUNCTIONS

In [9]:
def plot_example_errors(cls_pred,correct):
    incorrect = (correct==False)
    images = data.test.images[incorrect]
    cls_pred = cls_pred[incorrect]
    cls_true = data.test.cls[incorrect]
    plot_images(images[:9],cls_true[:9],cls_pred[:9])
    
    

def plot_confusion_matrix(cls_pred):
    cls_true = data.test.cls
    cm = confusion_matrix(y_true = cls_true , y_pred = cls_pred)
    print cm
    plt.imshow(cm,interpolation='nearest',cmap=plt.cm.Blues)
    
    plt.tight_layout()
    plt.colorbar()
    tick_marks = np.arange(num_classes)
    plt.xticks(tick_marks,range(num_classes))
    plt.yticks(tick_marks,range(num_classes))
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()
    
    

test_batch_size = 256
def print_accuracy(show_example_errors= False ,show_confusion_matrix = False) :
    num_test = len(data.test.images)
    cls_pred = np.zeros(shape=num_test,dtype = np.int)
    i=0
    while(i<num_test) :
        j=min(i + test_batch_size, num_test)
        images = data.test.images[i:j,:]
        labels = data.test.labels[i:j,:]
        feed_dict={x:images,y_true:labels}
        cls_pred[i:j] = session.run(y_pred_cls,feed_dict=feed_dict)
        i=j
        
    cls_true = data.test.cls
    correct = (cls_true==cls_pred)
    correct_sum = correct.sum()
    acc = float(correct_sum)/num_test
   
    print "Accuracy on Test-Set: {0:.1%} ({1} / {2})".format(acc, correct_sum, num_test)
    
    if show_example_errors:
        print("Example errors:")
        plot_example_errors(cls_pred=cls_pred, correct=correct)

    if show_confusion_matrix:
        print("Confusion Matrix:")
        plot_confusion_matrix(cls_pred=cls_pred)
    

#### Function to retrieve weights 
required when using prettyTensor

In [10]:
def get_pt_weights(layer_name):
    with tf.variable_scope(layer_name,reuse=True):
        variable = tf.get_variable('weights')
    return variable

# TENSORFLOW SESS RUN

In [11]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [12]:
print_accuracy()

Accuracy on Test-Set: 16.3% (1633 / 10000)


In [13]:
optimize(1)
print_accuracy()

Accuracy @iter      1 , Training Acc 0.234375
Total Time Elapsed : 0.245
Accuracy on Test-Set: 17.1% (1710 / 10000)


In [14]:
optimize(99)
print_accuracy()

Total Time Elapsed : 16.648
Accuracy on Test-Set: 86.1% (8607 / 10000)


In [15]:
total_iterations

100

In [16]:
optimize(100)
print_accuracy()

Accuracy @iter    101 , Training Acc 0.8203125
Total Time Elapsed : 16.633
Accuracy on Test-Set: 92.2% (9217 / 10000)


In [17]:
optimize(100)
print_accuracy()

Accuracy @iter    201 , Training Acc 0.8671875
Total Time Elapsed : 16.559
Accuracy on Test-Set: 93.9% (9386 / 10000)
